In [137]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from datetime import date
from datetime import timedelta
from datetime import datetime

%run web_scraping.ipynb
%run useful_functions.ipynb

In [138]:
shows = get_shows()
show_broadcast_details = get_show_broadcast_details()
broadcast_details = get_broadcast_details()
broadcast_playlists = get_broadcast_playlists()

In [141]:
#shows_to_run = ["https://www.bbc.co.uk/programmes/m00008zy/episodes/guide", 
#                "https://www.bbc.co.uk/programmes/b03hjfww/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b006wkh9/episodes/guide",
#                "https://www.bbc.co.uk/programmes/p04m6srg/episodes/guide",
#                "https://www.bbc.co.uk/programmes/m000m5bw/episodes/guide",
#                "https://www.bbc.co.uk/programmes/m0016zh5/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b0072lb2/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b0072q60/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b00p4l4t/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b0072l4x/episodes/guide"]

temp_show_broadcast_details = pd.DataFrame()

#for show_url in shows_to_run:
for show_url in shows['Show URL']:
    show_instance = Show(show_url, max_pages = 2)
    if shows.empty or show_instance.show_key not in shows['Show Key'].unique():
        shows = pd.concat([shows,show_instance.show_details])
    temp_show_broadcast_details = pd.concat([temp_show_broadcast_details,show_instance.broadcasts])
    
temp_show_broadcast_details['Datetime Key'] = datetime.now()
temp_show_broadcast_details['Is New Data'] = True
temp_show_broadcast_details['Is Maybe Pending'] = True
temp_show_broadcast_details.drop(columns=['Broadcast Number'], inplace=True)

show_broadcast_details = pd.concat([show_broadcast_details,temp_show_broadcast_details])
show_broadcast_details = show_broadcast_details.groupby(allbut(show_broadcast_details.columns,['Datetime Key','Is New Data','Is Maybe Pending'])).apply(lambda group: group.nsmallest(1, columns='Datetime Key')).reset_index(drop=True)
show_broadcast_details = show_broadcast_details.groupby("Broadcast Key").apply(lambda group: group.nlargest(1, columns='Datetime Key')).reset_index(drop=True)

In [134]:
max_trials = 50
new_broadcast_data = show_broadcast_details[(show_broadcast_details["Is Broadcast Upcoming"]==False) & ((show_broadcast_details["Is New Data"]==True) | (show_broadcast_details['Is Maybe Pending'] == True))][0:max_trials]
temp_broadcast_details = pd.DataFrame()
temp_broadcast_playlists = pd.DataFrame()

for i, [broadcast_url,broadcast_key] in enumerate(new_broadcast_data[['Broadcast URL','Broadcast Key']].values):
    broadcast_instance = Playlist(broadcast_url)
    if (i+1) % 5 == 0:
        print(f'Broadcast Number {1+i} of {len(new_broadcast_data.index)}')
    if (len(broadcast_instance.playlist.index) == 0) & ((pd.to_datetime('now')- broadcast_instance.broadcast_details["Service Date"]).dt.days.values <= 7):
        show_broadcast_details.loc[show_broadcast_details["Broadcast Key"] == broadcast_key,'Is Maybe Pending'] = True
    else:
        show_broadcast_details.loc[show_broadcast_details["Broadcast Key"] == broadcast_key,'Is Maybe Pending'] = False
    temp_broadcast_details = pd.concat([temp_broadcast_details,broadcast_instance.broadcast_details])
    temp_broadcast_playlists = pd.concat([temp_broadcast_playlists,broadcast_instance.playlist])
    
temp_broadcast_details['Datetime Key'] = datetime.now()
temp_broadcast_details['Is New Data'] = True
temp_broadcast_playlists['Datetime Key'] = datetime.now()
temp_broadcast_playlists['Is New Data'] = True

broadcast_details = pd.concat([broadcast_details,temp_broadcast_details])
broadcast_details = broadcast_details.groupby(allbut(broadcast_details.columns,['Datetime Key','Is New Data'])).apply(lambda group: group.nsmallest(1, columns='Datetime Key')).reset_index(drop=True)
broadcast_details = broadcast_details.groupby("Broadcast Key").apply(lambda group: group.nlargest(1, columns='Datetime Key')).reset_index(drop=True)

broadcast_playlists = pd.concat([broadcast_playlists,temp_broadcast_playlists])
broadcast_playlists = broadcast_playlists.groupby(allbut(broadcast_playlists.columns,['Datetime Key','Is New Data'])).apply(lambda group: group.nsmallest(1, columns='Datetime Key')).reset_index(drop=True)
broadcast_playlists = broadcast_playlists.groupby(["Broadcast Key", "Broadcast Position"]).apply(lambda group: group.nlargest(1, columns='Datetime Key')).reset_index(drop=True)


Broadcast Number 5 of 50
Broadcast Number 10 of 50
Broadcast Number 15 of 50
Broadcast Number 20 of 50
Broadcast Number 25 of 50
Broadcast Number 30 of 50
Broadcast Number 35 of 50
Broadcast Number 40 of 50
Broadcast Number 45 of 50
Broadcast Number 50 of 50


In [135]:
show_broadcast_details['Is New Data'] = False
broadcast_details['Is New Data'] = False
broadcast_playlists['Is New Data'] = False

shows.to_pickle("../data/shows.pkl")
show_broadcast_details.to_pickle("../data/show_broadcast_details.pkl")
broadcast_details.to_pickle("../data/broadcast_details.pkl")
broadcast_playlists.to_pickle("../data/broadcast_playlists.pkl")

In [136]:
mask_2022 = broadcast_details['Service Date'].apply(lambda date: date.year) == 2022

(
broadcast_details[mask_2022]
.merge(broadcast_playlists, on='Broadcast Key', how='left', suffixes=(" Broadcast Details",""))
.merge(show_broadcast_details, on='Broadcast Key', how='left', suffixes=(""," Show Details"))
.merge(shows, on='Show Key', how='left')
.sort_values(['Service Date','Broadcast Position'], ascending = [False, True])
)

,Show Title,Broadcast Banner,Broadcast Long Description,Service Image URL,Service Day,Service Date,Service Time,Broadcast Key,Datetime Key Broadcast Details,Is New Data Broadcast Details,...,Broadcast URL,Broadcast Title,Broadcast Short Description,Is Broadcast Upcoming,Show Key,Datetime Key Show Details,Is New Data Show Details,Is Maybe Pending,Show URL,Show Name
2018,Radio 1's Indie Show with Jack Saunders,The best in indie and alternative,Two indie fans go head to head in the ultimate...,https://programmes.files.bbci.co.uk/programmes...,Sun 04 Dec 2022,2022-12-04,21:00,-7514952971870662672,2022-12-11 13:15:55.124355,False,...,https://www.bbc.co.uk/programmes/m001fl2g,The best in indie and alternative,Two indie fans go head to head in a game of Ho...,False,8810658867441137647,2022-12-11 12:37:22.098745,False,False,https://www.bbc.co.uk/programmes/m00008zy/epis...,Radio 1's Indie Show with Jack Saunders
2019,Radio 1's Indie Show with Jack Saunders,The best in indie and alternative,Two indie fans go head to head in the ultimate...,https://programmes.files.bbci.co.uk/programmes...,Sun 04 Dec 2022,2022-12-04,21:00,-7514952971870662672,2022-12-11 13:15:55.124355,False,...,https://www.bbc.co.uk/programmes/m001fl2g,The best in indie and alternative,Two indie fans go head to head in a game of Ho...,False,8810658867441137647,2022-12-11 12:37:22.098745,False,False,https://www.bbc.co.uk/programmes/m00008zy/epis...,Radio 1's Indie Show with Jack Saunders
2020,Radio 1's Indie Show with Jack Saunders,The best in indie and alternative,Two indie fans go head to head in the ultimate...,https://programmes.files.bbci.co.uk/programmes...,Sun 04 Dec 2022,2022-12-04,21:00,-7514952971870662672,2022-12-11 13:15:55.124355,False,...,https://www.bbc.co.uk/programmes/m001fl2g,The best in indie and alternative,Two indie fans go head to head in a game of Ho...,False,8810658867441137647,2022-12-11 12:37:22.098745,False,False,https://www.bbc.co.uk/programmes/m00008zy/epis...,Radio 1's Indie Show with Jack Saunders
2021,Radio 1's Indie Show with Jack Saunders,The best in indie and alternative,Two indie fans go head to head in the ultimate...,https://programmes.files.bbci.co.uk/programmes...,Sun 04 Dec 2022,2022-12-04,21:00,-7514952971870662672,2022-12-11 13:15:55.124355,False,...,https://www.bbc.co.uk/programmes/m001fl2g,The best in indie and alternative,Two indie fans go head to head in a game of Ho...,False,8810658867441137647,2022-12-11 12:37:22.098745,False,False,https://www.bbc.co.uk/programmes/m00008zy/epis...,Radio 1's Indie Show with Jack Saunders
2022,Radio 1's Indie Show with Jack Saunders,The best in indie and alternative,Two indie fans go head to head in the ultimate...,https://programmes.files.bbci.co.uk/programmes...,Sun 04 Dec 2022,2022-12-04,21:00,-7514952971870662672,2022-12-11 13:15:55.124355,False,...,https://www.bbc.co.uk/programmes/m001fl2g,The best in indie and alternative,Two indie fans go head to head in a game of Ho...,False,8810658867441137647,2022-12-11 12:37:22.098745,False,False,https://www.bbc.co.uk/programmes/m00008zy/epis...,Radio 1's Indie Show with Jack Saunders
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1847,Radio 1's Future Artists with Jack Saunders,PinkPantheress co-host (Sound of 2022 winner),Jack co-hosts with the winner of The Sound of ...,https://programmes.files.bbci.co.uk/programmes...,Thu 6 Jan 2022,2022-01-06,20:00,-7675445680013112586,2022-12-11 13:15:55.124355,False,...,https://www.bbc.co.uk/programmes/m00130x7,PinkPantheress co-host (Sound of 2022 winner),Jack co-hosts with the winner of The Sound of ...,False,837400564190753462,2022-12-11 12:37:22.098745,False,False,https://www.bbc.co.uk/programmes/m000m5bw/epis...,Radio 1's Future Artists with Jack Saunders
1848,Radio 1's Future Artists with Jack Saunders,PinkPantheress co-host (Sound of 2022 winner),Jack co-hosts with the winner of The Sound of ...,https://programmes.files.bbci.co.uk/programmes...,Thu 6 Jan 2022

In [129]:
shows
show_broadcast_details
#broadcast_details
#broadcast_playlists

,Broadcast Image URL,Broadcast URL,Broadcast Title,Broadcast Short Description,Is Broadcast Upcoming,Broadcast Key,Show Key,Datetime Key,Is New Data,Is Maybe Pending
0,https://ichef.bbci.co.uk/images/ic/320x180/p0b...,https://www.bbc.co.uk/programmes/m00013fr,Today's Theme - The Over Driven Organ,Elbow frontman Guy Garvey loves the Hammond or...,False,-9217409020754396024,-3946016431430345654,2022-12-11 12:37:22.098745,False,False
1,https://ichef.bbci.co.uk/images/ic/320x180/p0c...,https://www.bbc.co.uk/programmes/m0009sbx,"New Music from Tame Impala, Pulled Apart by Ho...",Jack plays you the best in brand new indie and...,False,-9210574728566079079,8810658867441137647,2022-12-11 12:37:22.098745,False,False
2,https://ichef.bbci.co.uk/images/ic/320x180/p04...,https://www.bbc.co.uk/programmes/b091jbg4,Tom Ravenscroft sits in,"Chali 2na, Georgie Rogers and Eno Williams joi...",False,-9210072269533792936,-3239205012974803003,2022-12-11 12:37:22.098745,False,False
3,https://ichef.bbci.co.uk/images/ic/320x180/p0c...,https://www.bbc.co.uk/programmes/m0011zk7,Fickle Friends,Natti from Fickle Friends plays some of her al...,False,-9210012826913911293,8810658867441137647,2022-12-11 12:37:22.098745,False,False
4,https://ichef.bbci.co.uk/images/ic/320x180/p07...,https://www.bbc.co.uk/programmes/m000c3jr,Deathprod,"Stuart talks to Helge Sten, aka Deathprod",False,-9208310403030560793,-4306482833164206561,2022-12-11 12:37:22.098745,False,False
...,...,...,...,...,...,...,...,...,...,...
2731,https://ichef.bbci.co.uk/images/ic/320x180/p06...,https://www.bbc.co.uk/programmes/m00009bv,With Oliver Coates,A curiosity cabinet of experimental music; cel...,False,9197315014319359369,-4306482833164206561,2022-12-11 12:37:22.098745,False,True
2732,https://ichef.bbci.co.uk/images/ic/320x180/p0c...,https://www.bbc.co.uk/programmes/b0400fnn,"Britpop Roundtable: Jo Whiley, Matt Everitt, M...",Steve hosts a special Britpop Roundtable with ...,False,9204443453019991882,-3239205012974803003,2022-12-11 12:37:22.098745,False,True
2733,https://ichef.bbci.co.uk/images/ic/320x180/p09...,https://www.bbc.co.uk/programmes/m000ydd7,Sun Ra's Lanquidity,Light up your Freak beacon and step inside mus...,False,9208290808861630891,-4306482833164206561,2022-12-11 12:37:22.098745,False,True
2734,https://ichef.bbci.co.uk/images/ic/320x180/p0b...,https://www.bbc.co.uk/programmes/m001fv2v,14/12/2022,"The biggest and best new music in the world, i...",True,9208360684005449110,-8722995144322488383,2022-12-11 12:37:22.098745,False,True
